## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Import the dependencies.
import pandas as pd
import gmaps
import requests
import numpy as np
import matplotlib.pyplot as plt
# Import the API key.
from config import g_key

# Configure gmaps to use your Google API key.
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,Unnamed: 0,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Raudeberg,NO,61.9875,5.1352,45.27,72,100,5.01,overcast clouds
1,1,Solana Beach,US,32.9912,-117.2711,62.67,91,90,1.99,mist
2,2,Hoquiam,US,46.9809,-123.8893,52.93,65,90,9.22,moderate rain
3,3,Biak,ID,-0.9131,122.8766,77.40,76,98,2.24,overcast clouds
4,4,Bluff,NZ,-46.6000,168.3333,52.86,85,100,5.21,overcast clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 45
What is the maximum temperature you would like for your trip? 85


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
user_city_data_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
user_city_data_df.head(10)

,Unnamed: 0,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Raudeberg,NO,61.9875,5.1352,45.27,72,100,5.01,overcast clouds
1,1,Solana Beach,US,32.9912,-117.2711,62.67,91,90,1.99,mist
2,2,Hoquiam,US,46.9809,-123.8893,52.93,65,90,9.22,moderate rain
3,3,Biak,ID,-0.9131,122.8766,77.40,76,98,2.24,overcast clouds
4,4,Bluff,NZ,-46.6000,168.3333,52.86,85,100,5.21,overcast clouds
6,6,Westport,US,41.1415,-73.3579,58.89,58,90,11.50,overcast clouds
7,7,Ponta Do Sol,PT,32.6667,-17.1000,75.00,70,16,5.64,few clouds
10,10,Nemuro,JP,43.3236,145.5750,45.25,71,52,10.98,broken clouds
11,11,Georgetown,MY,5.4112,100.3354,81.90,86,40,0.00,moderate rain
12,12,Oskarshamn,SE,57.2645,16.4484,45.81,57,7,8.32,clear sky


In [5]:
# 4a. Determine if there are any empty rows.
user_city_data_df.count()

Unnamed: 0             465
City                   465
Country                460
Lat                    465
Lng                    465
Max Temp               465
Humidity               465
Cloudiness             465
Wind Speed             465
Current Description    465
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
user_city_data_df.dropna(inplace=True)


<ipython-input-6-a19fa93620b2>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  user_city_data_df.dropna(inplace=True)


In [7]:
user_city_data_df.count()

Unnamed: 0             460
City                   460
Country                460
Lat                    460
Lng                    460
Max Temp               460
Humidity               460
Cloudiness             460
Wind Speed             460
Current Description    460
dtype: int64

In [8]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = user_city_data_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Raudeberg,NO,45.27,overcast clouds,61.9875,5.1352,
1,Solana Beach,US,62.67,mist,32.9912,-117.2711,
2,Hoquiam,US,52.93,moderate rain,46.9809,-123.8893,
3,Biak,ID,77.40,overcast clouds,-0.9131,122.8766,
4,Bluff,NZ,52.86,overcast clouds,-46.6000,168.3333,
6,Westport,US,58.89,overcast clouds,41.1415,-73.3579,
7,Ponta Do Sol,PT,75.00,few clouds,32.6667,-17.1000,
10,Nemuro,JP,45.25,broken clouds,43.3236,145.5750,
11,Georgetown,MY,81.90,moderate rain,5.4112,100.3354,
12,Oskarshamn,SE,45.81,clear sky,57.2645,16.4484,


In [9]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

print("complete.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
complete.


In [10]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df['Hotel Name'].replace('', np.nan, inplace=True)

In [11]:
hotel_df.dropna(axis=0, how="any", inplace=True)

In [12]:
hotel_df.head()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Raudeberg,NO,45.27,overcast clouds,61.9875,5.1352,Nordre Silda.
1,Solana Beach,US,62.67,mist,32.9912,-117.2711,Hilton San Diego/Del Mar
2,Hoquiam,US,52.93,moderate rain,46.9809,-123.8893,Econo Lodge Inn & Suites
3,Biak,ID,77.40,overcast clouds,-0.9131,122.8766,Penginapan Kim
4,Bluff,NZ,52.86,overcast clouds,-46.6000,168.3333,Bluff Homestead - Guesthouse & Campervan Park


In [14]:
# 8a. Create the output File (CSV)
output_data_file = "../Vacation_Search/WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_data_file, index_label="City_ID")

In [15]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<d1>
<dt>City Name</dt><dd>{City}</dd>
<dt>Country Code</dt><dd>{Country}</dd>
<dt>Weather Description</dt><dd>{Current Description}</dd>
<dt>Maximum Temperature</dt><dd>{Max Temp}</dd>
</d1>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]
hotel_lat_lng = pd.DataFrame(locations)

In [16]:
hotel_lat_lng

,Lat,Lng
0,61.9875,5.1352
1,32.9912,-117.2711
2,46.9809,-123.8893
3,-0.9131,122.8766
4,-46.6000,168.3333
...,...,...
695,-10.4544,161.9205
696,38.9342,100.4517
697,-20.2833,44.2833
698,35.0456,-85.3097


In [17]:
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""
# Store the DataFrame Row.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

In [18]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)



# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))